In [1]:
#Mcallen
#San Antonio
#Edinburg
#Austin
#Dallas
#San Marcos
#Corpus Christi
#Weslaco
#Boerne
#Houston

### Imports

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import urllib.parse

### HTTP Request

In [2]:
#store website in a variable
website = 'https://www.trulia.com/TX/San_Antonio/'

In [3]:
#set up selenium webdriver
driver = webdriver.Chrome()

#get the webpage using selenium

try:
    # Get the webpage using Selenium
    driver.get(website)
    time.sleep(5)  # Give the page some time to load

    # Scroll down to load more content
    for _ in range(20):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Adjust the delay based on the page loading speed

    # Get the updated page source after scrolling
    updated_html = driver.page_source

    # Create a BeautifulSoup object using the updated HTML
    soup = BeautifulSoup(updated_html, 'html.parser')

    # Extract the desired results with data-testid
    result_container = soup.find_all(attrs={'data-testid': True})


    # Retrieve anchor tags into a list and establish base URL
    url_part_1 = 'https://www.trulia.com'
    url_part_2 = []

    for item in result_container:
        for link in item.find_all('div', {'data-testid': 'property-card-details'}):
            url_part_2.append(link.find('a').get('href'))

finally:
    driver.quit()  # Close the browser window when done

In [4]:
# Remove spaces and duplicates from list
url_only = [x.strip(' ') for x in url_part_2]
url_only = list(dict.fromkeys(url_only))
len(url_only)

40

In [5]:
# Concatenate URL
url_joined = []
for link_2 in url_only:
    url_joined.append(urllib.parse.urljoin(url_part_1,link_2))

In [6]:
url_joined

['https://www.trulia.com/home/2755-knoll-tree-st-san-antonio-tx-78247-26392212',
 'https://www.trulia.com/home/7738-watersedge-cv-san-antonio-tx-78254-169471994',
 'https://www.trulia.com/home/220-greenlawn-dr-san-antonio-tx-78201-26241783',
 'https://www.trulia.com/home/475-salz-way-san-antonio-tx-78260-333541889',
 'https://www.trulia.com/home/15319-fallen-grv-san-antonio-tx-78247-26461840',
 'https://www.trulia.com/home/3928-claret-cup-san-antonio-tx-78253-118215546',
 'https://www.trulia.com/home/8406-lajitas-bnd-san-antonio-tx-78254-169468284',
 'https://www.trulia.com/home/318-ashland-dr-san-antonio-tx-78218-26296491',
 'https://www.trulia.com/home/9850-raven-field-dr-san-antonio-tx-78245-26153212',
 'https://www.trulia.com/home/26-marbella-ct-san-antonio-tx-78257-89614077',
 'https://www.trulia.com/home/4931-bernadine-dr-san-antonio-tx-78220-26314295',
 'https://www.trulia.com/home/118-perch-mnr-san-antonio-tx-78253-83991500',
 'https://www.trulia.com/home/119-Spofford-St-11-San

### Get data from links

In [9]:
# address
# bedrooms
# bathrooms
# sqft
# year built
# parking
# price

In [7]:
first_link = url_joined[0]
print(first_link)

https://www.trulia.com/home/2755-knoll-tree-st-san-antonio-tx-78247-26392212


### Get request and Soup Object

In [8]:
response = requests.get(first_link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})

In [9]:
soup = BeautifulSoup(response.content, 'html.parser')

In [1]:
# Commenting this step out to avoid copyright
#soup

#### Put all of the attributes together and loop through results from page 1


In [11]:
address = []
bedrooms = []
bathrooms = []
area = []
year_built = []
parking = []
price = []

# Loop through all joined links
for link in url_joined:
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})
    
    # Create soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Address
    try:
        address.append(soup.find('span', {'data-testid':'home-details-summary-headline'}).get_text())
    except:
        address.append('')
    # Bedrooms  
    try:
        bedrooms.append(soup.find('li', {'data-testid':'bed'}).get_text())
    except:
        bedrooms.append('')
        
    # Bathrooms
    try:
        bathrooms.append(soup.find('li', {'data-testid':'bath'}).get_text())
    except:
        bathrooms.append('')
        
    # Square Feet
    try:
        area.append(soup.find('li', {'data-testid':'floor'}).get_text())
    except:
        area.append('')
    
    # Year Built
    try:
        year_built.append(soup.find('div',string='Year Built').findNext('span').get_text())
        year_built = [x.strip('Year Built: ') for x in year_built] 
    except:
        year_built.append('')
    
    # Parking
    try:
        parking.append(soup.find(string='Parking').findNext('div').get_text())
    except:
        parking.append('')
    
    # Price
    try:
        price.append(soup.find('h3', {'data-testid':'on-market-price-details'}).get_text())
    except:
        price.append('') 
    
    # Create a dictionary with results
    output = {'Address': address, 'Bathrooms': bathrooms, 'Area':area, 'Year Built':year_built, 
              'Parking':parking, 'Price':price}
    
output

{'Address': ['2755 Knoll Tree',
  '7738 Watersedge Cv',
  '220 GREENLAWN Dr',
  '475 SALZ WAY',
  '15319 FALLEN GRV',
  '3928 CLARET CUP',
  '8406 LAJITAS BND',
  '318 ASHLAND DR',
  '9850 raven field',
  '26 MARBELLA CT',
  '4931 BERNADINE DR',
  '118 PERCH MANOR',
  '119 Spofford St LOT 11',
  '2507 SILVER RIDGE ST',
  'Parker Plan in Davis Ranch by Coventry Homes',
  '14022 Samuel Blair',
  '122 Pierce Ave LOT 3',
  '2900 SAN PEDRO AVE',
  '7834 Broadway UNIT 704',
  '8003 Radiant Sunset',
  '806 NOGALITOS ST LOT 2',
  'Gemini Plan in Morgan Meadows by CastleRock Communities',
  '.17 ACRE ON ADA ST LOT 2',
  '8255 Misty Willow',
  '4723 HIGHLAND FARM',
  '1001 CAPITOL AVE',
  '826 LOMBRANO ST',
  '7926 VANITY HILL',
  'Homes Available Soon Plan in Spring Creek by Century Communities',
  'Ashlyn Plan in Highland Estates by Sitterle Homes',
  '4463 TAYLORS BND',
  "2594W Plan in Kallison Ranch 45' by PERRY HOMES",
  'The Nicole Plan in Preserve at Medina by D.R. Horton - San Antonio',

In [13]:
# Create dataframe
df = pd.DataFrame(output)
df

,Address,Bathrooms,Area,Year Built,Parking,Price
0,2755 Knoll Tree,3 Baths,"1,584 sqft",1979,2 Car Garage,"$249,900"
1,7738 Watersedge Cv,2 Baths,,2017,Garage,"$280,000"
2,220 GREENLAWN Dr,2 Baths,"1,879 sqft (on 0.47 acres)",1928,Garage,"$360,000"
3,475 SALZ WAY,3 Baths,,2021,Garage,"$410,000"
4,15319 FALLEN GRV,3 Baths,"2,345 sqft (on 0.32 acres)",1986,2 Car Garage,"$409,900"
5,3928 CLARET CUP,3 Baths,"2,523 sqft",2012,Garage,"$517,000"
6,8406 LAJITAS BND,2 Baths,"2,325 sqft (on 0.27 acres)",2016,2 Car Garage,"$495,000"
7,318 ASHLAND DR,1 Bath,927 sqft,1955,Garage,"$160,000"
8,9850 raven field,2 Baths,"2,192 sqft",1992,2 Car Garage,"$270,000"
9,26 MARBELLA CT,3 Baths,"2,732 sqft",2015,Garage,"$845,000"


In [14]:
# Add location to dataframe
df['Location'] = 'San Antonio'
df

,Address,Bathrooms,Area,Year Built,Parking,Price,Location
0,2755 Knoll Tree,3 Baths,"1,584 sqft",1979,2 Car Garage,"$249,900",San Antonio
1,7738 Watersedge Cv,2 Baths,,2017,Garage,"$280,000",San Antonio
2,220 GREENLAWN Dr,2 Baths,"1,879 sqft (on 0.47 acres)",1928,Garage,"$360,000",San Antonio
3,475 SALZ WAY,3 Baths,,2021,Garage,"$410,000",San Antonio
4,15319 FALLEN GRV,3 Baths,"2,345 sqft (on 0.32 acres)",1986,2 Car Garage,"$409,900",San Antonio
5,3928 CLARET CUP,3 Baths,"2,523 sqft",2012,Garage,"$517,000",San Antonio
6,8406 LAJITAS BND,2 Baths,"2,325 sqft (on 0.27 acres)",2016,2 Car Garage,"$495,000",San Antonio
7,318 ASHLAND DR,1 Bath,927 sqft,1955,Garage,"$160,000",San Antonio
8,9850 raven field,2 Baths,"2,192 sqft",1992,2 Car Garage,"$270,000",San Antonio
9,26 MARBELLA CT,3 Baths,"2,732 sqft",2015,Garage,"$845,000",San Antonio


In [15]:
df.to_excel('dataframe_san_antonio.xlsx', index=False)